In [2]:
import pandas as pd
import pickle
import os
%pip install pyportfolioopt


     |████████████████████████████████| 61kB 4.1MB/s 
     |████████████████████████████████| 1.3MB 8.5MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for cvxpy: filename=cvxpy-1.1.11-cp37-cp37m-linux_x86_64.whl size=2725497 sha256=c67c4e3810ca35591aba4f5b824246cedf15a9e9c863872067d178943f1ad5a1
  Stored in directory: /root/.cache/pip/wheels/df/d4/eb/c74e09a39bc4ac09b3a8d2a48c1c6009f391056d8c87bf68aa
Successfully built cvxpy
  Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


In [4]:
from pypfopt.expected_returns import mean_historical_return,ema_historical_return
from  pypfopt.risk_models import CovarianceShrinkage
from  pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df = pd.DataFrame()

In [7]:

for f in os.listdir('/content/drive/MyDrive/colab/BSE_df'):
  with open(f'/content/drive/MyDrive/colab/BSE_df/{f}','rb') as file:
    if df.empty:
      df = pickle.load(file)['Adj Close']
    else:
      df = pd.concat([df,pickle.load(file)['Adj Close']],axis=1)
    
    

In [8]:
df.columns = [i.strip('.csv') for i in os.listdir('/content/drive/MyDrive/colab/BSE_df')]

In [9]:
df.head()

,ASIANPAINT.BO,AXISBANK.BO,BAJAJ-AUTO.BO,BAJFINANCE.BO,BAJAJFINSV.BO,BHARTIARTL.BO,DRREDDY.BO,HCLTECH.BO,HDFC.BO,HDFCBANK.BO,HINDUNILVR.BO,ICICIBANK.BO,INDUSINDBK.BO,INFY.BO,ITC.BO,KOTAKBANK.BO,LT.BO,M&M.BO,MARUTI.BO,NESTLEIND.BO,NTPC.BO,ONGC.BO,POWERGRID.BO,RELIANCE.BO,SBIN.BO,SUNPHARMA.BO,TCS.BO,TECHM.BO,TITAN.BO,ULTRACEMCO.BO
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-01-01,1122.931519,564.149414,3021.046387,1716.279053,5176.046875,472.922546,2357.325684,425.996277,1606.401489,914.870544,1290.398438,304.259735,1605.153931,474.939514,235.195282,1000.599915,1179.642334,726.202637,9282.260742,7405.251465,123.481689,164.395981,171.722458,897.702820,307.049988,558.157654,1240.125732,461.942505,839.659485,4221.885742
2018-01-02,1119.356445,559.005859,3021.734619,1712.202637,5105.479980,462.311646,2353.610840,431.170166,1620.396362,922.605286,1282.927368,302.986084,1622.328369,472.621124,233.406403,1001.548828,1167.769165,733.021484,9166.729492,7413.126953,125.126701,168.283737,171.936783,897.555115,302.899994,556.114258,1230.362427,470.903748,831.593445,4219.456543
2018-01-03,1117.642578,559.655029,2978.896484,1704.397827,5098.388672,464.236877,2283.716797,433.636200,1619.301514,912.727478,1293.128052,308.668610,1609.151611,467.846710,233.451126,999.001526,1193.057861,735.115845,9043.420898,7409.212402,123.901703,165.506760,173.008438,902.429871,302.850006,553.924866,1231.039429,477.153717,842.364563,4258.577148
2018-01-04,1147.956421,558.606323,2992.931152,1743.272949,5047.499023,467.818573,2355.370117,431.774628,1623.680786,915.018372,1294.995972,308.423645,1624.943970,466.102234,233.138062,999.251282,1229.752197,735.456726,9010.288086,7423.172363,123.901703,170.334427,171.851044,906.270630,308.049988,565.358459,1237.859985,478.348511,877.137268,4304.638672
2018-01-05,1159.415771,562.401611,3006.507568,1800.989502,5120.912598,483.533447,2411.920654,435.207733,1643.149658,916.570190,1299.258179,306.268250,1676.466797,464.633148,235.150543,1001.998352,1228.910889,736.723083,9060.034180,7395.017578,124.146698,168.924576,172.236816,908.978882,306.200012,563.169067,1254.046875,481.427490,894.203918,4358.780762


In [11]:
dd = df.iloc[:400]
dz = df.iloc[400:]
print('amount required to buy 1 sahre of each company :',dz.loc[dz.index.min()].sum())

amount required to buy 1 sahre of each company : 57965.08702087402


In [17]:

mu = ema_historical_return(dd)
s = CovarianceShrinkage(dd).ledoit_wolf()
ef = EfficientFrontier(mu,s,weight_bounds=(0,1))
wts = ef.max_sharpe()
clean_wt = ef.clean_weights()
# print(clean_wt)
ef.portfolio_performance(verbose=True)
latest_price = get_latest_prices(dd)
da = DiscreteAllocation(clean_wt, latest_price, total_portfolio_value=58000)
print(da.lp_portfolio())
  

Expected annual return: 35.4%
Annual volatility: 13.5%
Sharpe Ratio: 2.48
({'ASIANPAINT.BO': 5, 'BAJFINANCE.BO': 3, 'ICICIBANK.BO': 12, 'INFY.BO': 11, 'KOTAKBANK.BO': 3, 'NESTLEIND.BO': 1, 'POWERGRID.BO': 12, 'TCS.BO': 4}, 64.2164306640625)


In [13]:
a = (dz.loc[dz.index.max()].sum() - dz.loc[dz.index.min()].sum())/dz.loc[dz.index.min()].sum()
print('market return over 382 days : ' + str(a*100) + ' %')
print('annual market return : ' + str(((1+a)**0.759 - 1)*100) + ' %')

market return over 382 days : 41.80052958371859 %
annual market return : 30.353738256121844 %


In [14]:
min_val = 0
max_val = 0
for key in da.lp_portfolio()[0]:
  min_val = da.lp_portfolio()[0][key]*dz.loc[dz.index.min(),key] + min_val
  max_val = da.lp_portfolio()[0][key]*dz.loc[dz.index.max(),key] + max_val
print('portfolio return over 382 days : ' + str((max_val/min_val - 1)*100) + ' %')  
print('annual portfolio return : ' + str(((max_val/min_val)**0.759 - 1)*100) + ' %')

portfolio return over 382 days : 51.5512977125429 %
annual portfolio return : 37.10231297230071 %


In [18]:
mu = ema_historical_return(df)
s = CovarianceShrinkage(df).ledoit_wolf()
ef = EfficientFrontier(mu,s,weight_bounds=(0,1))
wts = ef.max_sharpe()
clean_wt = ef.clean_weights()
# print(clean_wt)
ef.portfolio_performance(verbose=True)
latest_price = get_latest_prices(df)
da = DiscreteAllocation(clean_wt, latest_price, total_portfolio_value=200000,short_ratio=0.3)
print(da.lp_portfolio())

Expected annual return: 65.3%
Annual volatility: 22.1%
Sharpe Ratio: 2.87
({'ASIANPAINT.BO': 2, 'BAJFINANCE.BO': 4, 'DRREDDY.BO': 1, 'HCLTECH.BO': 28, 'INFY.BO': 46, 'POWERGRID.BO': 85, 'SBIN.BO': 52, 'ULTRACEMCO.BO': 6}, 243.19747924804688)
